### General setup

In [2]:
%matplotlib widget

In [3]:
import fiftyone as fo
import fiftyone.zoo as foz

In [46]:
import pandas as pd
from random import shuffle

Set download location

In [4]:
import pathlib
datasets_path = pathlib.Path().resolve() / "raw_datasets"
fo.config.dataset_zoo_dir = datasets_path

List avaialble datasets

In [5]:
fo.list_datasets()

['bdd100k-train',
 'bdd_train_6000',
 'coco-2017-train-12000',
 'kitti-train',
 'kitti_train_6000']

## Raw dataset preparation

If Raw datasets already preapred "Raw dataset preparation" can be skipped - move to "TF Record CReation"

### KITTI Dataset

#### Training and Validation

In [51]:
kitti_train = foz.load_zoo_dataset("kitti", split="train")

Split 'train' already downloaded
Loading existing dataset 'kitti-train'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


Extract 20% from training set for validation

In [57]:
index = int(len(kitti_train) * 0.8)
kitti_train_view = kitti_train.shuffle()[:index]
kitti_validation_view = kitti_train.shuffle()[index:]

Check if all images are used and none is duplicated (due to rounding of index)

In [59]:
len(kitti_train_view) + len(kitti_validation_view) == len(kitti_train)

True

In [62]:
kitti_train_6000 = fo.Dataset()
kitti_train_6000.merge_samples(kitti_train_view)
kitti_train_6000.name = 'kitti_train_6000'

#### Test

In [ ]:
kitti_test = foz.load_zoo_dataset("kitti", split="test")

Dataset operations

In [5]:
view = kitti_train.view()

In [ ]:
view.take(len(view)).first()

In [ ]:
x = view.take(10).first()
gt = x.ground_truth
gt.detections[0]
x.filepath

### COCO Dataset

In [49]:
def download_coco(split, max_samples):
    # Restarting download after connection timeout
    try:
        coco_train = fo.zoo.load_zoo_dataset("coco-2017", split=split, label_types=["detections"], classes=["person", "bicycle", "car", "motorcycle", "bus", "train", "truck", 
        "traffic light", "fire hydrant", "parking meter", "bench", "cat", "dog", "chair", "couch", "bed", "dining table", "tv", "laptop"], only_matching=True, max_samples=max_samples)
    except:
        coco_train = download_coco(split, max_samples)

    return coco_train

In [50]:
coco_train = download_coco('train', 12000)

Found annotations at 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\coco-2017\raw\instances_train2017.json'
11798 images found; downloading the remaining 202
 100% |██████████████████| 202/202 [22.0s elapsed, 0s remaining, 10.0 images/s]     
Writing annotations for 12000 downloaded samples to 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\coco-2017\train\labels.json'
Dataset info written to 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\coco-2017\info.json'
Loading 'coco-2017' split 'train'
 100% |█████████████| 12000/12000 [44.4s elapsed, 0s remaining, 228.1 samples/s]      
Dataset 'coco-2017-train-12000' created


In [ ]:
coco_validation = download_coco('validation', 2400)

In [ ]:
coco_test = download_coco('test', 2400)

### BDD Dataset

#### Training

In [4]:
bdd_train = foz.load_zoo_dataset(
    "bdd100k",
    split="train",
    source_dir=r'raw_datasets/bdd100k/',
)

Preparing split 'train' in 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\bdd100k\train'
Preparing training images...
Preparing training labels...
Preparing validation images...
Preparing validation labels...
Preparing test images...
Parsing dataset metadata
Found 70000 samples
Dataset info written to 'D:\_moje\AGH\Magisterka\Master-Thesis\model_training\OD\data\raw_datasets\bdd100k\info.json'
Loading 'bdd100k' split 'train'
 100% |█████████████| 70000/70000 [1.0h elapsed, 0s remaining, 42.3 samples/s]      
Dataset 'bdd100k-train' created


Take only 6000 images from shuffled dataset

In [17]:
bdd_train_view = bdd_train.view()
bdd_train_view = bdd_train_view.shuffle()[:6000]

Save view as a new dataset

In [46]:
bdd_train_6000 = fo.Dataset()
bdd_train_6000.merge_samples(bdd_train_view)
bdd_train_6000.name = 'bdd_train_6000'

#### Validation

In [ ]:
bdd_validation = foz.load_zoo_dataset(
    "bdd100k",
    split="validation",
    source_dir=r'raw_datasets/bdd100k/',
)

#### Test

In [ ]:
bdd_test = foz.load_zoo_dataset(
    "bdd100k",
    split="test",
    source_dir=r'raw_datasets/bdd100k/',
)

## Load datasets from disk

No need to run this if "Raw dataset preparation" was run in this session

In [6]:
kitti_train_6000 = fo.load_dataset('kitti_train_6000')
bdd_train_6000 = fo.load_dataset('bdd_train_6000')
coco_train = fo.load_dataset('coco-2017-train-12000')

## TF Record Creation

Need to preproces data only once

### Preprocess data

Delete images without labels and containing only "traffic sign" label from BDD Dataset

In [117]:
dropped_ids = []
for img in bdd_train_6000:
    if img.detections:
        det_labels = []
        for det in img.detections.detections:
            det_labels.append(det.label)
        det_labels = set(det_labels)
        if len(det_labels) == 1 and "traffic sign" in det_labels:
            dropped_ids.append(img.id)
    else:
        dropped_ids.append(img.id)

In [ ]:
bdd_train_6000.delete_samples(dropped_ids)
bdd_train_6000.save()

Delete images without labels and with labels containig only "Misc" or "DontCare" from KITTI dataset

In [150]:
correct_labels = ["Car", "Pedestrian", "Van", "Cyclist", "Truck", "Tram", "Person_sitting"]

In [159]:
dropped_ids = []
for img in kitti_train_6000:
    det_labels = []
    for det in img.ground_truth.detections:
        det_labels.append(det.label)
    for label in correct_labels:
        if label in det_labels:
            break
        dropped_ids.append(img.id)

In [155]:
kitti_train_6000.delete_samples(dropped_ids)
kitti_train_6000.save()

### Datasets statistics

In [8]:
bdd_labels = []
for img in bdd_train_6000:
    for det in img.detections.detections:
        bdd_labels.append(det.label)

In [9]:
kitti_labels = []
for img in kitti_train_6000:
    for det in img.ground_truth.detections:
        kitti_labels.append(det.label)

In [10]:
coco_labels = []
for img in coco_train:
    for det in img.ground_truth.detections:
        coco_labels.append(det.label)

In [11]:
bdd_labels = pd.DataFrame({"bdd": bdd_labels})

In [12]:
bdd_labels["bdd"].value_counts()

car              62162
traffic sign     20027
traffic light    15479
person            7273
truck             2592
bus               1004
bike               625
rider              376
motor              251
train               21
Name: bdd, dtype: int64

In [13]:
kitti_labels = pd.DataFrame({"kitti": kitti_labels})

In [14]:
kitti_labels["kitti"].value_counts()

Car               22938
DontCare           8236
Van                2219
Pedestrian         1914
Cyclist            1078
Truck               845
Misc                767
Tram                328
Person_sitting       84
Name: kitti, dtype: int64

In [15]:
coco_labels = pd.DataFrame({"coco": coco_labels})

In [16]:
coco_labels["coco"].value_counts()

person           35278
car               6050
chair             5027
dining table      2065
traffic light     1740
truck             1358
bench             1277
motorcycle        1132
bicycle            873
dog                770
bus                768
couch              725
tv                 719
laptop             660
train              635
cat                621
bed                578
fire hydrant       253
parking meter      179
Name: coco, dtype: int64

### Changing labels

In [17]:
df = pd.DataFrame({"ids": [11, 11, 23, 23, 11, 32, 4, 5], "values": [0 , 1, 2, 3, 4, 5, 6,7]})

In [18]:
df["ids"].unique()

array([11, 23, 32,  4,  5], dtype=int64)

In [19]:
df.query("ids == 11")

,ids,values
0,11,0
1,11,1
4,11,4


Map datasets default labels to expected labels:

        1 - person
        2 - car
        3 - big_car
        4 - bike
        5 - train
        6 - traffic_light
        7 - animal
        8 - obstacle


In [33]:
class DFHandler:
    def prepare_input(self, data, ids, filenames, bounding_boxes, labels):
        for bbox, label in zip(data['bboxes'], data['labels']):
            ids.append(data["id"])
            filenames.append(data['filename'])
            bounding_boxes.append(bbox)
            labels.append(label)

        return ids, filenames, bounding_boxes, labels

    def handle(self, dataset, ids, filenames, bounding_boxes, labels):
        for img in dataset:
            data = self.preprocess_data(img)
            ids, filenames, bounding_boxes, labels = self.prepare_input(data, ids, filenames, bounding_boxes, labels)

        return ids, filenames, bounding_boxes, labels




In [34]:
class COCOHandler(DFHandler):
    label_map = {
        "person": 1,
        "car": 2,
        "chair": 8,
        "dining table": 8,
        "traffic light": 6,
        "truck": 3,
        "bench": 8,
        "motorcycle": 4,
        "bicycle": 4,
        "dog": 7,
        "bus": 3,
        "couch": 8,
        "tv": 8,
        "laptop": 8,
        "train": 5,
        "cat": 7,
        "bed": 8,
        "fire hydrant": 8,
        "parking meter": 8,
    }

    def preprocess_data(self, data):
        labels = []
        bboxes = []
        for detection in data.ground_truth.detections:
            bboxes.append(detection.bounding_box)
            labels.append(self.label_map[detection.label])

        return {"id": f"{data['id']}_coco", 'filename': data["filepath"], "labels": labels, "bboxes":bboxes}

In [35]:
class BDDHandler(DFHandler):
    label_map = {
        'traffic light': 6,
        'car': 2,
        'person': 1,
        'motor': 4,
        'bus': 3,
        'truck': 3,
        'bike': 4,
        'train': 5,
    }

    def preprocess_data(self, data):
        labels = []
        bboxes = []
        for detection in data.detections.detections:
            if detection.label in self.label_map:
                bboxes.append(detection.bounding_box)
                labels.append(self.label_map[detection.label])

        return {"id": f"{data['id']}_bdd", 'filename': data["filepath"], "labels": labels, "bboxes":bboxes}

In [36]:
class KITTIHandler(DFHandler):
    label_map = {
        'Car': 2,
        'Van': 3,
        'Pedestrian': 1,
        'Truck': 3,
        'Cyclist': 4,
        'Tram': 5,
        'Person_sitting': 1
    }

    def preprocess_data(self, data):
        labels = []
        bboxes = []
        for detection in data.ground_truth.detections:
            if detection.label in self.label_map:
                bboxes.append(detection.bounding_box)
                labels.append(self.label_map[detection.label])

        return {"id": f"{data['id']}_kitti", 'filename': data["filepath"], "labels": labels, "bboxes":bboxes}

In [37]:
coco_handler = COCOHandler()
bdd_handler = BDDHandler()
kitti_handler = KITTIHandler()

In [38]:
id, filename, boxes, labels = coco_handler.handle(coco_train, [], [], [], [])
id, filename, boxes, labels = bdd_handler.handle(bdd_train_6000, id, filename, boxes, labels)
id, filename, boxes, labels = kitti_handler.handle(kitti_train_6000, id, filename, boxes, labels)

In [39]:
dataset_df = pd.DataFrame({"id": id,"filename": filename, "bboxes": boxes, "label": labels})

In [52]:
ids = dataset_df["id"].unique()
ids = shuffle(ids)

In [45]:
dataset_df.query(f'id == "{ids[2]}"')

,id,filename,bboxes,label
2,634e9fbc42bd0e0a4cf573bd_coco,D:\_moje\AGH\Magisterka\Master-Thesis\model_tr...,"[0.5338320209973753, 0.52086, 0.17241469816272...",1
3,634e9fbc42bd0e0a4cf573bd_coco,D:\_moje\AGH\Magisterka\Master-Thesis\model_tr...,"[0.31083989501312337, 0.52264, 0.1493700787401...",1
4,634e9fbc42bd0e0a4cf573bd_coco,D:\_moje\AGH\Magisterka\Master-Thesis\model_tr...,"[0.3132283464566929, 0.66842, 0.03133858267716...",1
5,634e9fbc42bd0e0a4cf573bd_coco,D:\_moje\AGH\Magisterka\Master-Thesis\model_tr...,"[0.7462992125984251, 0.6668, 0.028556430446194...",1
6,634e9fbc42bd0e0a4cf573bd_coco,D:\_moje\AGH\Magisterka\Master-Thesis\model_tr...,"[0.5013123359580053, 0.66874, 0.01879265091863...",1
7,634e9fbc42bd0e0a4cf573bd_coco,D:\_moje\AGH\Magisterka\Master-Thesis\model_tr...,"[0.9101312335958005, 0.6668, 0.038845144356955...",1
